在上一课中，将数据管道纳入了交易算法。现在是时候定义算法如何将使用流水线生成的数据来重新平衡投资组合。我们的目标是根据尾部12个月股利与股价比率找到最大化回报的目标投资组合，同时保持由一组规则或约束条件定义的特定结构。这通常被称为投资组合优化问题。

`Optimize API`使我们能够轻松地将管道输出转化为目标和一组约束条件。 然后，我们可以使用`order_optimal_portfolio`将当前的投资组合转换为符合规范的目标投资组合。

第一步是定义一个目标。我们将使用`MaximizeAlpha`，它将尝试将资金分配给与尾部12个月股利与股价比率成比例的资产。

接下来，我们需要指定我们希望目标投资组合满足的约束列表。 我们首先在初始化时定义一些阈值并将它们存储在我们的上下文变量中：

现在，我们可以使用我们上面定义的阈值来指定`rebalance`中的约束条件：

最后，我们可以将我们的目标和约束列表传递给`order_optimal_portfolio`以计算目标投资组合，并发布将我们当前投资组合转换为最佳状态所需的订单：

# 风险管理

除了对目标投资组合的结构设置限制外，我们还希望限制可能影响其业绩的共同风险因子敞口。例如，由于stocktwits的情绪数据具有短暂性，我们打算利用情绪分数高峰期，算法可能会面临短期逆转风险。

我们将使用Quantopian的风险模型来管理我们投资组合的共同风险因子。风险模型计算资产的16种不同风险因子敞口：11个部门因子和5个主题因子（包括短期逆转）。我们可以使用`risk_loading_pipeline`函数在算法中使用这些数据，该函数返回一个数据管道，该数据管道为风险模型中的每个因子生成一列输出。

与数据管道类似，需要将风险数据管道附加到我们的算法，并提供一个名称来识别它。 然后，在`before_trading_start`中获取它的输出并将其存储在`context`中：

下一步是将`RiskModelExposure`约束添加到我们的投资组合优化逻辑。 该约束采用风险模型生成的数据，并对模型中包含的每个因子设置目标投资组合的整体风险敞口限制。

最后，以下算法包含策略和投资组合构建逻辑，并准备好进行回测。

本部分主要测试`risk_loading_pipeline`和`RiskModelExposure`优化问题求解，相对于原案例有较大改动。

In [1]:
%load_ext zipline

In [2]:
%%zipline --start 2018-1-8 --end 2018-6-8

# Import Algorithm API functions
from zipline.api import (
    attach_pipeline,
    pipeline_output,
    order_optimal_portfolio,
    schedule_function,
    date_rules,
    time_rules,
)

# Import Optimize API module
import zipline.optimize as opt

# Pipeline imports
from zipline.pipeline import Pipeline
#from zipline.pipeline.data.psychsignal import stocktwits
from zipline.pipeline.builtin import QTradableStocksUS, trailing_dividend_yield
from zipline.pipeline.factors import SimpleMovingAverage

# 运行时函数在zipline.pipeline.builtin模块
#from zipline.pipeline.filters import QTradableStocksUS
from zipline.pipeline.risk import risk_loading_pipeline

import numpy as np
import pandas as pd


def make_pipeline():

    base_universe = QTradableStocksUS()

    # 尾部12个月每股股利平均值/股价
    tdy = trailing_dividend_yield()
    # 只是利用了Index作为股票总体
    return Pipeline(
        columns={
            'tdy': tdy,
        },
        # Set screen as the intersection between our filter
        # and trading universe
        screen=(
            base_universe
            & tdy.notnull()  # 选择非空的那一部分与可交易总体的交集
        ))

def initialize(context):
    # Constraint parameters
    context.max_leverage = 1.5 # 尽管限制总杠杆为1.5，由于算法尚未考虑cash因素，所以部分日期会有一定程度的偏离
    context.max_pos_size = 0.015
    context.max_turnover = 0.95

    # Attach data pipelines
    attach_pipeline(
        make_pipeline(),
        'data_pipe'
    )
    attach_pipeline(
        risk_loading_pipeline(),
        'risk_pipe'
    )

    # Schedule rebalance function
    schedule_function(
        rebalance,
        date_rules.week_start(),
        time_rules.market_open(),
    )

def handle_data(context, data):
    t = context.get_datetime('Asia/Shanghai')
    print(t, '股票数量与绝对值权重合计')
    s = context.get_current_allocations()
    print(s.abs().sum())
    print(s)

def before_trading_start(context, data):
    # Get pipeline outputs and
    # store them in context
    output = pipeline_output('data_pipe')
    
    context.alphas = pd.Series(np.random.random_integers(-100,100, len(output)) / 100,
                               index = output.index)
    context.risk_factor_betas = pipeline_output('risk_pipe').fillna(0.0)



def rebalance(context, data):
    # Create MaximizeAlpha objective using
    # tdy data from pipeline output
    objective = opt.MaximizeAlpha(
      context.alphas
    )

    # Create position size constraint
    constrain_pos_size = opt.PositionConcentration.with_equal_bounds(
        -context.max_pos_size,
        context.max_pos_size
    )
    
    # Ensure long and short books
    # are roughly the same size
    #dollar_neutral = opt.DollarNeutral()

    # Constrain target portfolio's leverage
    max_leverage = opt.MaxGrossExposure(context.max_leverage)

    # Constrain portfolio turnover
    # 没有实现周转率限制
    # max_turnover = opt.MaxTurnover(context.max_turnover)

    # Constrain target portfolio's risk exposure
    # By default, max sector exposure is set at
    # 0.2, and max style exposure is set at 0.4
    factor_risk_constraints = opt.RiskModelExposure(
        context.risk_factor_betas,
        #version=opt.Newest # 弃用版本参数
    )

    # Rebalance portfolio using objective
    # and list of constraints
    order_optimal_portfolio(
        objective=objective,
        constraints=[
            max_leverage,
            #dollar_neutral,
            #constrain_pos_size,
            factor_risk_constraints,
        ]
    )


2018-01-08 15:00:00+08:00 股票数量与绝对值权重合计
0.0
Series([], dtype: float64)
2018-01-09 15:00:00+08:00 股票数量与绝对值权重合计
1.4967362083508713
特 力Ａ(000025)    -0.066820
铜陵有色(000630)    -0.081042
华讯方舟(000687)    -0.075897
中兴商业(000715)    -0.047019
凯撒旅游(000796)     0.032420
超声电子(000823)    -0.018675
云南铜业(000878)     0.025650
金智科技(002090)    -0.029245
*ST凡谷(002194)   -0.033092
联化科技(002250)     0.030651
中超控股(002471)    -0.041863
惠博普(002554)      0.019209
江苏国信(002608)    -0.033272
康弘药业(002773)    -0.020988
金刚玻璃(300093)     0.032028
腾邦国际(300178)     0.024364
中威电子(300270)    -0.028162
北信源(300352)     -0.057187
红相股份(300427)     0.020770
金雷风电(300443)     0.025423
新美星(300509)     -0.056257
华舟应急(300527)    -0.026903
雄帝科技(300546)    -0.077956
华能国际(600011)    -0.060601
九鼎投资(600053)    -0.087276
南京高科(600064)     0.093006
航天动力(600343)     0.029548
大有能源(600403)     0.039609
精工钢构(600496)     0.029863
新华医疗(600587)     0.020584
万业企业(600641)    -0.030020
外高桥(600648)     -0.041081
宏发股份(600885)    -0.016966
文峰股份(601010)  

2018-01-23 15:00:00+08:00 股票数量与绝对值权重合计
1.608800932792224
红 太 阳(000525)   -0.120724
太阳能(000591)     -0.050088
创维数字(000810)     0.042567
*ST中基(000972)   -0.042179
宏润建设(002062)     0.017951
广百股份(002187)     0.032239
大华股份(002236)     0.078764
龙蟒佰利(002601)     0.031254
露笑科技(002617)    -0.031261
华宏科技(002645)     0.028373
天际股份(002759)    -0.028632
易联众(300096)      0.030515
联建光电(300269)     0.060373
扬帆新材(300637)     0.010449
林海股份(600099)     0.088695
澳柯玛(600336)     -0.040170
亚宝药业(600351)     0.074003
首开股份(600376)     0.142970
香梨股份(600506)     0.021441
新安股份(600596)     0.038339
山西焦化(600740)    -0.143399
创业环保(600874)    -0.061038
博瑞传播(600880)    -0.195767
依顿电子(603328)    -0.037429
珍宝岛(603567)      0.030941
歌力思(603808)      0.129241
dtype: float64
2018-01-24 15:00:00+08:00 股票数量与绝对值权重合计
1.6407083518960288
红 太 阳(000525)   -0.120401
太阳能(000591)     -0.049892
创维数字(000810)     0.042306
*ST中基(000972)   -0.042175
宏润建设(002062)     0.018139
广百股份(002187)     0.032959
大华股份(002236)     0.078289
龙蟒佰利(002601)

2018-02-13 15:00:00+08:00 股票数量与绝对值权重合计
1.5414801755185148
南 玻Ａ(000012)     0.035075
深天马Ａ(000050)     0.064255
丽珠集团(000513)    -0.031674
潍柴重机(000880)    -0.057885
华菱钢铁(000932)    -0.027349
中泰化学(002092)     0.061111
科大讯飞(002230)    -0.051019
杰瑞股份(002353)    -0.037023
同德化工(002360)    -0.038786
力生制药(002393)    -0.064567
星网锐捷(002396)    -0.034460
利源精制(002501)     0.030976
金达威(002626)      0.107535
金发拉比(002762)    -0.038462
瑞尔特(002790)      0.015791
经纬电材(300120)     0.034719
宝利国际(300135)    -0.074427
常山药业(300255)    -0.041463
富春股份(300299)     0.047014
云意电气(300304)     0.059344
普丽盛(300442)      0.021490
维宏股份(300508)     0.049242
浙江富润(600070)    -0.060861
宋都股份(600077)     0.036024
岷江水电(600131)    -0.057195
光电股份(600184)    -0.063837
*ST成城(600247)   -0.046263
凤竹纺织(600493)    -0.056168
国睿科技(600562)     0.034479
上海电气(601727)     0.034608
XD爱普股(603020)    0.032972
天龙股份(603266)     0.035769
康德莱(603987)      0.059639
dtype: float64
2018-02-14 15:00:00+08:00 股票数量与绝对值权重合计
1.5161473663802054
南 玻Ａ(000012

2018-03-13 15:00:00+08:00 股票数量与绝对值权重合计
1.573382852265059
斯太尔(000760)      0.186543
天保基建(000965)     0.063788
粤 水 电(002060)   -0.089220
山河智能(002097)     0.047290
恒宝股份(002104)    -0.063180
银轮股份(002126)    -0.100337
威华股份(002240)     0.039748
瑞康医药(002589)     0.063258
首航节能(002665)    -0.066950
汇洁股份(002763)    -0.079874
鹭燕医药(002788)     0.052108
永和智控(002795)    -0.048688
黄山胶囊(002817)    -0.051085
英唐智控(300131)     0.046970
银信科技(300231)     0.056098
温州宏丰(300283)    -0.046220
艾比森(300389)     -0.044840
有研新材(600206)    -0.078431
宁波韵升(600366)    -0.067805
莫高股份(600543)     0.077235
东材科技(601208)    -0.105397
京运通(601908)     -0.045875
鼎信通讯(603421)     0.052441
dtype: float64
2018-03-14 15:00:00+08:00 股票数量与绝对值权重合计
1.5502868277393345
斯太尔(000760)      0.179170
天保基建(000965)     0.063389
粤 水 电(002060)   -0.088460
山河智能(002097)     0.047293
恒宝股份(002104)    -0.061357
银轮股份(002126)    -0.099747
威华股份(002240)     0.039785
瑞康医药(002589)     0.063387
首航节能(002665)    -0.066820
汇洁股份(002763)    -0.077878
鹭燕医药(002788)

2018-04-03 15:00:00+08:00 股票数量与绝对值权重合计
1.558959889435905
中国宝安(000009)     0.069276
鄂武商Ａ(000501)     0.029734
苏宁环球(000718)    -0.044864
岳阳兴长(000819)     0.095886
天保基建(000965)    -0.160339
红 宝 丽(002165)    0.052909
众业达(002441)      0.056915
捷顺科技(002609)    -0.071865
常山药业(300255)     0.070032
博雅生物(300294)     0.063441
裕兴股份(300305)     0.065313
南华仪器(300417)     0.074163
唐德影视(300426)    -0.047550
葛洲坝(600068)      0.082964
营口港(600317)     -0.056143
盘江股份(600395)    -0.125770
航天长峰(600855)    -0.030305
贵阳银行(601997)     0.125275
奥康国际(603001)     0.059131
海汽集团(603069)    -0.080646
九洲药业(603456)     0.031689
盛洋科技(603703)    -0.064753
dtype: float64
2018-04-04 15:00:00+08:00 股票数量与绝对值权重合计
1.5512591386346524
中国宝安(000009)     0.069729
鄂武商Ａ(000501)     0.029494
苏宁环球(000718)    -0.044208
岳阳兴长(000819)     0.095267
天保基建(000965)    -0.158430
红 宝 丽(002165)    0.052243
众业达(002441)      0.056323
捷顺科技(002609)    -0.071475
常山药业(300255)     0.071168
博雅生物(300294)     0.066587
裕兴股份(300305)     0.065004
南华仪器(300417)

2018-04-24 15:00:00+08:00 股票数量与绝对值权重合计
1.4203666916278292
*ST宜化(000422)    0.028708
一汽轿车(000800)     0.060547
华东医药(000963)     0.054604
德美化工(002054)    -0.038037
宏达高科(002144)     0.061975
延华智能(002178)    -0.045734
友阿股份(002277)    -0.103430
顺灏股份(002565)     0.072406
亿利达(002686)      0.050206
威星智能(002849)    -0.039141
易世达(300125)     -0.072857
捷成股份(300182)     0.051166
汉邦高科(300449)     0.104967
晨化股份(300610)     0.098049
青山纸业(600103)     0.037050
宁沪高速(600377)     0.027373
东百集团(600693)     0.124441
上柴股份(600841)    -0.044787
中炬高新(600872)     0.034893
长城汽车(601633)     0.041018
浙江仙通(603239)     0.078591
华铁科技(603300)    -0.040969
信捷电气(603416)    -0.054150
再升科技(603601)    -0.055270
dtype: float64
2018-04-25 15:00:00+08:00 股票数量与绝对值权重合计
1.5470714677505681
*ST宜化(000422)    0.027311
一汽轿车(000800)     0.059963
华东医药(000963)     0.053914
德美化工(002054)    -0.038082
宏达高科(002144)     0.062046
延华智能(002178)    -0.044996
友阿股份(002277)    -0.104726
顺灏股份(002565)     0.071997
亿利达(002686)      0.050593
威星智能(002849

2018-05-15 15:00:00+08:00 股票数量与绝对值权重合计
1.5151322820380986
珠海港(000507)     0.042474
厦门信达(000701)   -0.016055
福晶科技(002222)    0.045104
科华恒盛(002335)    0.060599
达实智能(002421)   -0.057907
金新农(002548)    -0.020133
博彦科技(002649)    0.105242
华统股份(002840)   -0.045049
机器人(300024)    -0.085345
元力股份(300174)    0.035652
远方信息(300306)    0.165773
鼎捷软件(300378)   -0.048827
安硕信息(300380)   -0.071884
富祥股份(300497)   -0.082188
汇金科技(300561)    0.081048
神宇股份(300563)    0.054791
天铁股份(300587)   -0.052884
恒锋信息(300605)   -0.058946
双良节能(600481)   -0.090621
中化国际(600500)   -0.026309
智慧能源(600869)   -0.080492
四方冷链(603339)    0.052490
拉芳家化(603630)   -0.070356
晨光文具(603899)    0.064962
dtype: float64
2018-05-16 15:00:00+08:00 股票数量与绝对值权重合计
1.5042510658285484
珠海港(000507)     0.042765
厦门信达(000701)   -0.016043
福晶科技(002222)    0.044249
科华恒盛(002335)    0.056654
达实智能(002421)   -0.057736
博彦科技(002649)    0.103084
华统股份(002840)   -0.045206
机器人(300024)    -0.084289
元力股份(300174)    0.035601
远方信息(300306)    0.164823
鼎捷软件(300378)   -0.0

2018-06-05 15:00:00+08:00 股票数量与绝对值权重合计
1.454559115445492
深南电A(000037)   -0.037204
苏常柴Ａ(000570)    0.054667
贝瑞基因(000710)    0.041231
国元证券(000728)    0.047935
潍柴重机(000880)   -0.036442
华联股份(000882)    0.049038
华峰氨纶(002064)   -0.070164
宁波东力(002164)    0.067136
金风科技(002202)    0.041922
扬子新材(002652)   -0.058982
龙泉股份(002671)    0.086591
长生生物(002680)    0.036733
红日药业(300026)    0.041737
高新兴(300098)     0.037877
锐奇股份(300126)    0.079760
日照港(600017)    -0.043959
巨化股份(600160)    0.038926
北方股份(600262)    0.000481
科达洁能(600499)    0.043037
祥源文化(600576)    0.130584
申达股份(600626)    0.065062
南京化纤(600889)   -0.043091
唐山港(601000)    -0.044239
中海油服(601808)   -0.052815
汇嘉时代(603101)    0.064245
碳元科技(603133)    0.071938
诺邦股份(603238)   -0.032810
欧普照明(603515)    0.035953
dtype: float64
[2018-06-11 23:04:11.003796] INFO: Performance: after split: asset: 巨化股份(600160), amount: 45983, cost_basis: 8.67, last_sale_price: 11.27
[2018-06-11 23:04:11.005045] INFO: Performance: returning cash: 5.2
2018-06-06 15:00:00+08

,algo_volatility,algorithm_period_return,alpha,benchmark_period_return,benchmark_volatility,beta,capital_used,ending_cash,ending_exposure,ending_value,...,short_exposure,short_value,shorts_count,sortino,starting_cash,starting_exposure,starting_value,trading_days,transactions,treasury_period_return
2018-01-08 07:00:00+00:00,NaN,0.000000,NaN,0.005173,NaN,NaN,0.000000e+00,1.000000e+07,0.000,0.000,...,0.000000e+00,0.000000e+00,0,NaN,1.000000e+07,0.000,0.000,1,[],0.0
2018-01-09 07:00:00+00:00,0.010236,-0.000912,0.649251,0.012213,0.020553,-0.498046,4.445591e+06,1.444559e+07,-4454710.250,-4454710.250,...,-9.704212e+06,-9.704212e+06,21,-11.224972,1.000000e+07,0.000,0.000,2,"[{'amount': -17144, 'dt': 2018-01-09 07:00:00+...",0.0
2018-01-10 07:00:00+00:00,0.056575,0.004749,3.416712,0.016687,0.021096,-2.164298,2.171226e+04,1.446730e+07,-4419812.142,-4419812.142,...,-9.641905e+06,-9.641905e+06,21,47.782250,1.444559e+07,-4454710.250,-4454710.250,3,"[{'amount': -1051, 'dt': 2018-01-10 07:00:00+0...",0.0
2018-01-11 07:00:00+00:00,0.092017,-0.003733,-1.467592,0.016149,0.051101,1.220296,0.000000e+00,1.446730e+07,-4504629.710,-4504629.710,...,-9.705740e+06,-9.705740e+06,21,-3.447000,1.446730e+07,-4419812.142,-4419812.142,4,[],0.0
2018-01-12 07:00:00+00:00,0.081064,-0.002565,-1.420602,0.020840,0.044459,1.241113,0.000000e+00,1.446730e+07,-4492953.410,-4492953.410,...,-9.663923e+06,-9.663923e+06,21,-2.103156,1.446730e+07,-4504629.710,-4504629.710,5,[],0.0
2018-01-15 07:00:00+00:00,0.112927,0.010258,0.559240,0.020897,0.047756,-0.143432,0.000000e+00,1.446730e+07,-4364724.670,-4364724.670,...,-9.390949e+06,-9.390949e+06,21,7.892550,1.446730e+07,-4492953.410,-4492953.410,6,[],0.0
2018-01-16 07:00:00+00:00,0.110129,0.005475,0.715716,0.028927,0.050995,-0.499020,-3.244637e+06,1.122267e+07,-1167918.030,-1167918.030,...,-7.874731e+06,-7.874731e+06,17,3.459969,1.446730e+07,-4364724.670,-4364724.670,7,"[{'amount': 17144, 'dt': 2018-01-16 07:00:00+0...",0.0
2018-01-17 07:00:00+00:00,0.106769,0.000604,0.013161,0.026425,0.059728,0.013136,-1.780513e+04,1.120486e+07,-1198822.180,-1198822.180,...,-7.925519e+06,-7.925519e+06,16,0.393551,1.122267e+07,-1167918.030,-1167918.030,8,"[{'amount': 1883, 'dt': 2018-01-17 07:00:00+00...",0.0
2018-01-18 07:00:00+00:00,0.099960,0.001485,0.021592,0.032055,0.057085,0.027526,0.000000e+00,1.120486e+07,-1190007.390,-1190007.390,...,-7.937277e+06,-7.937277e+06,16,0.800215,1.120486e+07,-1198822.180,-1198822.180,9,[],0.0
2018-01-19 07:00:00+00:00,0.095372,-0.001251,-0.057161,0.035432,0.053834,0.033738,0.000000e+00,1.120486e+07,-1217371.560,-1217371.560,...,-7.899510e+06,-7.899510e+06,16,-0.488448,1.120486e+07,-1190007.390,-1190007.390,10,[],0.0
